#### Szempont opciók (görgess lejjebb a mit_fozzek függvényhez, ahol ki van írva, hogy EZT MÓDOSÍTSD): 
##### 1 - legtöbb akciós termék
##### 2 - legnagyobb akciós arány a termékek számán belül
##### 3 - legnagyobb megspórolt összeg
##### 4 - legkevesebb elkészítési idő

In [7]:
#Használt package-ek beimportálása
import requests
import bs4
from tqdm import tqdm
import json

#Az Aldi akciós lista összerakásához:
akcios_dict = []
akcio_idotartam = ''
base_url = 'https://www.aldi.hu/hu/ajanlatok/mindig-akcio/'
aldi_link = requests.get(base_url)
aldi_oldal = bs4.BeautifulSoup(aldi_link.text, 'lxml')
minden_info = aldi_oldal.select('.ratio-container.m-ratio-57x25.box--description.m-no-ratio-on-phone')
for i in tqdm(range(0, len(minden_info))):
    d = {}
    nev = minden_info[i].select('.box--description--header')
    tiszta_nev = [[tag for tag in szo.children][-1].strip() for szo in nev]
    d['termek_neve'] = tiszta_nev[0]
    if 'Akciók' in tiszta_nev[0]:
        akcio_idotartam = tiszta_nev[0]
    elif 'Új termékek' in tiszta_nev[0]:
        akcio_idotartam = tiszta_nev[0]
    d['akcio_idotartama'] = akcio_idotartam
    er_ar_bonyi = minden_info[i].select('.box--former-price')
    er_ar = [mat.text.strip() for mat in er_ar_bonyi]
    eredeti_ar = []
    szamos_er_ar = 0
    for l in er_ar:
        l = l.replace(' Ft', '')
        l = l.replace(' ', '')
        l = l.replace(',', '')
        eredeti_ar.append(int(l))
        szamos_er_ar = int((eredeti_ar)[0])
    d['eredeti_ar'] = szamos_er_ar
    akc_ar_bonyi = minden_info[i].select('.box--value')
    akc_ar = [mat.text.strip() for mat in akc_ar_bonyi]
    akcios_ar = []
    szamos_akc_ar = 0
    for k in akc_ar:
        if k != 'Ft':
            k = k.replace(' ', '')
            k = k.replace(',', '')
            akcios_ar.append(int(k))
            szamos_akc_ar = int((akcios_ar)[0])
    if 'Akciók' in tiszta_nev[0]:
        pass
    elif 'Új termék' in tiszta_nev[0]:
        pass
    elif er_ar == []:
        pass
    else:
        if ',' in er_ar[0]:
            szamos_akc_ar = (szamos_akc_ar*100 + 90)
    d['akcios_ar'] = szamos_akc_ar
    d['sporolas'] = szamos_er_ar - szamos_akc_ar
    akcios_dict.append(d)
    
#Aldi akciósok kiexportálása
with open('aldi_akcios.json', 'w', encoding='utf-8') as f: json.dump(akcios_dict, f, ensure_ascii=False, indent=4)
        
#Nosalty-s receptek linkjének kiexportálása
nosalty_url = "https://www.nosalty.hu/kereses/recept/egyszer%C5%B1?mikor[0]=ebed&mikor[1]=vacsora&fokategoria=egytaletelek&page={}.html"
href = []
for j in range(1,11):
    res = requests.get(nosalty_url.format(j))
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    nosalty = soup.select('.m-articleCard__headline.-smallArticle.a-link.mb-6')
    for l in range(0, len(nosalty)):
        hosszu = nosalty[l]
        ref = hosszu.get('href')
        href.append(ref)
with open('href.txt', 'w') as file:
    for a in href:
        file.write(a + '\n')
    
#Nosaltys receptek kigyűjtése és akciós termékek jelölgetése
receptkonyv = []
for i in tqdm(range(0, len(href))):
    d = {}
    base_url = 'https://www.nosalty.hu{}'
    recept_link = requests.get(base_url.format(href[i]))
    recept_oldal = bs4.BeautifulSoup(recept_link.text, 'lxml')
    recept_nev = recept_oldal.select('h1')
    recept = [mat.text.strip() for mat in recept_nev][0]
    d['recept_nev'] = recept
    d['recept_link'] = base_url.format(href[i])
    hozzavalok_bonyi = recept_oldal.select('.a-link.-primaryHoverEffect.-fontSize-16')
    hozzavalok = [mat.text.strip() for mat in hozzavalok_bonyi]
    hozzavalok = list(dict.fromkeys(hozzavalok))
    akcios_termek_szama = 0
    hozzavalok_jelolt = []
    sporolt_osszeg = 0
    for termek in hozzavalok:
        hozzaadando = termek
        for k in range(0, len(akcios_dict)):
            if termek in akcios_dict[k]['termek_neve'].lower():
                akcios_termek_szama = akcios_termek_szama + 1
                hozzaadando = "* " + termek + " * " + akcios_dict[k]['akcio_idotartama']
                sporolt_osszeg = sporolt_osszeg + akcios_dict[k]['sporolas']
                break
        hozzavalok_jelolt.append(hozzaadando)
    d['hozzavalok'] = list(dict.fromkeys(hozzavalok_jelolt))
    d['akcios_termek_szama'] = akcios_termek_szama
    d['akcios_arany'] = akcios_termek_szama / len(hozzavalok)
    d['sporolt_osszeg'] = sporolt_osszeg
    ido_bonyi = recept_oldal.select('time')
    ido = [mat.text.strip() for mat in ido_bonyi][3]
    d['elkeszitesi_ido'] = ido+' perc'
    receptkonyv.append(d)

#Függvény
def mit_fozzek(szempont, topszam):
    if szempont == 1:
        uj = {}
        for i in receptkonyv:
            uj[("\033[1m" + i['recept_nev'] + "\033[0m" + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link'])]=i['akcios_termek_szama']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        while k<topszam:
            toprendezett = rendezett[k]
            print(toprendezett[0] + '\n')
            k=k+1
    if szempont == 2:
        uj = {}
        for i in receptkonyv:
            uj[("\033[1m" + i['recept_nev'] + "\033[0m" + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link'])]=i['akcios_arany']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        while k<topszam:
            toprendezett = rendezett[k]
            print(toprendezett[0] + '\n')
            k=k+1
    if szempont == 3:
        uj = {}
        for i in receptkonyv:
            uj[("\033[1m" + i['recept_nev'] + "\033[0m" + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link'])]=i['sporolt_osszeg']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        while k<topszam:
            toprendezett = rendezett[k]
            print(toprendezett[0] + '\n')
            k=k+1
    if szempont == 4:
        uj = {}
        for i in receptkonyv:
            uj[("\033[1m" + i['recept_nev'] + "\033[0m" + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link'])]=i['elkeszitesi_ido']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        while k<topszam:
            toprendezett = rendezett[k]
            print(toprendezett[0] + '\n')
            k=k+1

100%|████████████████████████████████████████████████████████████████████████████████| 193/193 [06:28<00:00,  2.01s/it]


In [16]:
#EZT MÓDOSÍTD!!!!
mit_fozzek(2, 3)

Egyszerű rakott krumpli
̲H̲o̲z̲z̲á̲v̲a̲l̲ó̲k̲:
* burgonya * Akciók vasárnaptól szerdáig, 2022.01.30-tól 2022.02.02-ig
* kolbász * Akciók csütörtöktől szerdáig, 2022.01.27-től 2022.02.02-ig
bacon
* tejföl * Akciók csütörtöktől szerdáig, 2022.01.27-től 2022.02.02-ig
tojás
* tej * Akciók csütörtöktől szerdáig, 2022.01.27-től 2022.02.02-ig
só
bors
Spórolt összeg: 470 Ft
Elkészítési idő: 90 perc
Link: https://www.nosalty.hu/recept/egyszeru-rakott-krumpli

Kolbászos-káposztás egytál
̲H̲o̲z̲z̲á̲v̲a̲l̲ó̲k̲:
* burgonya * Akciók vasárnaptól szerdáig, 2022.01.30-tól 2022.02.02-ig
savanyú káposzta
* kolbász * Akciók csütörtöktől szerdáig, 2022.01.27-től 2022.02.02-ig
fehérbor
* víz * Akciók csütörtöktől szerdáig, 2022.01.27-től 2022.02.02-ig
só
Spórolt összeg: 410 Ft
Elkészítési idő: 60 perc
Link: https://www.nosalty.hu/recept/kolbaszos-kaposztas-egytal

Mindent bele egytálétel
̲H̲o̲z̲z̲á̲v̲a̲l̲ó̲k̲:
* burgonya * Akciók vasárnaptól szerdáig, 2022.01.30-tól 2022.02.02-ig
* csirkecomb * Akciók csütö

In [86]:
import smtplib

In [10]:
#E-mail küldéshez:
def mit_fozzek_e(szempont, topszam):
    if szempont == 1:
        uj = {}
        for i in receptkonyv:
            uj[(i['recept_nev'].upper() + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link']) + "\n"]=i['akcios_termek_szama']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        vegso = ''
        while k<topszam:
            toprendezett = rendezett[k]
            valamelyik = toprendezett[0] + '\n'
            vegso = vegso + valamelyik
            k=k+1
        return vegso
    if szempont == 2:
        uj = {}
        for i in receptkonyv:
            uj[(i['recept_nev'].upper() + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link']) + "\n"]=i['akcios_arany']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        vegso = ''
        while k<topszam:
            toprendezett = rendezett[k]
            valamelyik = toprendezett[0] + '\n'
            vegso = vegso + valamelyik
            k=k+1
        return vegso
    if szempont == 3:
        uj = {}
        for i in receptkonyv:
            uj[(i['recept_nev'].upper() + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link']) + "\n"]=i['sporolt_osszeg']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        vegso = ''
        while k<topszam:
            toprendezett = rendezett[k]
            valamelyik = toprendezett[0] + '\n'
            vegso = vegso + valamelyik
            k=k+1
        return vegso
    if szempont == 4:
        uj = {}
        for i in receptkonyv:
            uj[(i['recept_nev'].upper() + "\u0332".join("\nHozzávalók:") + "\n" +
            "\n".join(i['hozzavalok']) + "\nSpórolt összeg: " + str(i['sporolt_osszeg']) + " Ft" +
            "\nElkészítési idő: " + i['elkeszitesi_ido'] +
            "\nLink: " + i['recept_link']) + "\n"]=i['elkeszitesi_ido']
        rendezett=sorted(uj.items(), key=lambda x: x[1], reverse=True)
        k=0
        vegso = ''
        while k<topszam:
            toprendezett = rendezett[k]
            valamelyik = toprendezett[0] + '\n'
            vegso = vegso + valamelyik
            k=k+1
        return vegso

In [14]:
#TÖLTSD KI A KÍVÁNT ÉRTÉKEKKEL
emailre = mit_fozzek_e(1, 5)

In [ ]:
#TÖLTSD KI A KÍVÁNT ÉRTÉKEKKEL
emailre = mit_fozzek_e(szempont, topszam)

In [15]:
import smtplib
import getpass
smtp_object = smtplib.SMTP('smtp.gmail.com', 587)
smtp_object.ehlo()
smtp_object.starttls()
email = getpass.getpass('Email: ')
password = getpass.getpass('Password: ')
smtp_object.login(email, password)
from_address = email
to_address = email
subject = "Heti olcsó recept ajánlat"
msg = "Subject: "+subject+"\n" + emailre
message = msg.encode('utf-8')

smtp_object.sendmail(from_address, to_address, message)

Email: ········
Password: ········


{}